d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 1200px">
</div>

# Advanced Keras

Congrats on building your first neural network! In this notebook, we will cover even more topics to improve your model building. After you learn the concepts here, you will apply them to the neural network you just created.

We will use the California Housing Dataset.

## ![Spark Logo Tiny](https://files.training.databricks.com/images/105/logo_spark_tiny.png) In this lesson you:<br>
 - Perform data standardization for better model convergence
 - Add validation data
 - Generate model checkpointing/callbacks
 - Use TensorBoard
 - Apply dropout regularization

In [3]:
%run "./Includes/Classroom-Setup"

In [4]:
from sklearn.datasets.california_housing import fetch_california_housing
from sklearn.model_selection import train_test_split

cal_housing = fetch_california_housing()

# split 80/20 train-test
X_train, X_test, y_train, y_test = train_test_split(cal_housing.data,
                                                    cal_housing.target,
                                                    test_size=0.2,
                                                    random_state=1)

print(cal_housing.DESCR)

Downloading Cal. housing from https://ndownloader.figshare.com/files/5976036 to /root/scikit_learn_data
.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

 :Number of Instances: 20640

 :Number of Attributes: 8 numeric, predictive attributes and the target

 :Attribute Information:
 - MedInc median income in block
 - HouseAge median house age in block
 - AveRooms average number of rooms
 - AveBedrms average number of bedrooms
 - Population block population
 - AveOccup average house occupancy
 - Latitude house block latitude
 - Longitude house block longitude

 :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bureau publishes sample data (a block group typically has a population
of 600 to 3,000 people).

It can be downloaded/loaded using the
:func:`sklearn.datasets.fetch_california_housing` function.

.. topic:: References

 - Pace, R. Kelley and Ronald Barry, Sparse Spatial Autoregressions,
 Statistics and Probability Letters, 33 (1997) 291-297

Let's take a look at the distribution of our features.

In [6]:
import pandas as pd

pd.DataFrame(X_train, columns=cal_housing.feature_names).describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
count,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000,16512.000000
mean,3.876149,28.604469,5.441114,1.099598,1425.257146,3.094971,35.632194,-119.574288
std,1.891584,12.586046,2.613727,0.507173,1123.756792,11.597402,2.137087,2.007578
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.750000,32.540000,-124.300000
25%,2.572050,18.000000,4.439906,1.006260,786.000000,2.427283,33.930000,-121.810000
50%,3.544550,29.000000,5.226528,1.048797,1164.000000,2.813449,34.260000,-118.490000
75%,4.750000,37.000000,6.057778,1.099574,1723.000000,3.273834,37.710000,-118.010000
max,15.000100,52.000000,141.909091,34.066667,35682.000000,1243.333333,41.950000,-114.310000


## 1. Data Standardization

Because our features are all on different scales, it's going to be more difficult for our neural network during training. Let's do feature-wise standardization.

We are going to use the [StandardScaler](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) from Sklearn, which will remove the mean (zero-mean) and scale to unit variance.

$$x' = \frac{x - \bar{x}}{\sigma}$$

In [8]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Keras Model

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense  
tf.random.set_seed(42)

model = Sequential([
  Dense(20, input_dim=8, activation="relu"),
  Dense(20, activation="relu"),
  Dense(1, activation="linear")
])

#creating linear stack of layers
##each layer has their own AF

In [11]:
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(lr=0.01), loss="mse", metrics=["mse"])

## 2. Validation Data

Let's take a look at the [.fit()](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential#fit) method in the docs to see all of the options we have available! 

We can either explicitly specify a validation dataset, or we can specify a fraction of our training data to be used as our validation dataset.

The reason why we need a validation dataset is to evaluate how well we are performing on unseen data (neural networks will overfit if you train them for too long!).

We can specify `validation_split` to be any value between 0.0 and 1.0 (defaults to 0.0).

In [13]:
history = model.fit(X_train, y_train, validation_split=.2, epochs=10, verbose=2)

Epoch 1/10
413/413 - 1s - loss: 0.6822 - mse: 0.6822 - val_loss: 0.4361 - val_mse: 0.4361
Epoch 2/10
413/413 - 1s - loss: 0.3911 - mse: 0.3911 - val_loss: 0.3821 - val_mse: 0.3821
Epoch 3/10
413/413 - 1s - loss: 0.3647 - mse: 0.3647 - val_loss: 0.3767 - val_mse: 0.3767
Epoch 4/10
413/413 - 1s - loss: 0.3620 - mse: 0.3620 - val_loss: 0.3699 - val_mse: 0.3699
Epoch 5/10
413/413 - 1s - loss: 0.3490 - mse: 0.3490 - val_loss: 0.3643 - val_mse: 0.3643
Epoch 6/10
413/413 - 1s - loss: 0.3408 - mse: 0.3408 - val_loss: 0.3558 - val_mse: 0.3558
Epoch 7/10
413/413 - 1s - loss: 0.3286 - mse: 0.3286 - val_loss: 0.3634 - val_mse: 0.3634
Epoch 8/10
413/413 - 1s - loss: 0.3263 - mse: 0.3263 - val_loss: 0.3380 - val_mse: 0.3380
Epoch 9/10
413/413 - 1s - loss: 0.3157 - mse: 0.3157 - val_loss: 0.4185 - val_mse: 0.4185
Epoch 10/10
413/413 - 1s - loss: 0.3235 - mse: 0.3235 - val_loss: 0.3265 - val_mse: 0.3265

## 3. Checkpointing

After each epoch, we want to save the model. However, we will pass in the flag `save_best_only=True`, which will only save the model if the validation loss decreased. This way, if our machine crashes or we start to overfit, we can always go back to the "good" state of the model.

To accomplish this, we will use the ModelCheckpoint [callback](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint). History is an example of a callback that is automatically applied to every Keras model.

In [15]:
from tensorflow.keras.callbacks import ModelCheckpoint

filepath = f"{working_dir}/keras_checkpoint_weights.ckpt"

model_checkpoint = ModelCheckpoint(filepath=filepath, verbose=1, save_best_only=True)

In [16]:
working_dir

Out[12]: '/dbfs/user/odl_user_195841@databrickslabs.onmicrosoft.com/deep_learning/dl_03_advanced_keras'

## 4. Tensorboard

[Tensorboard](https://www.tensorflow.org/tensorboard/get_started) provides a nice UI to visualize and debug your neural networks. We can also define it as a callback.

In [18]:
from tensorflow.keras.callbacks import TensorBoard

log_dir = f"{working_dir}/_tb.dir"
tensorboard = TensorBoard(log_dir)

dbutils.tensorboard.start(log_dir) # Will be empty until call .fit() below

#dbutils is a databricks utility

TensorBoard log directory set to: /dbfs/user/odl_user_195841@databrickslabs.onmicrosoft.com/deep_learning/dl_03_advanced_keras/_tb.dir. 
 View TensorBoard

Now let's add in our model checkpoint and Tensorboard callbacks to our `.fit()` command.

In [20]:
history = model.fit(X_train, y_train, validation_split=.2, epochs=10, verbose=2, callbacks=[model_checkpoint, tensorboard])

Epoch 1/10
WARNING:tensorflow:Method (on_train_batch_end) is slow compared to the batch update (0.116791). Check your callbacks.

Epoch 00001: val_loss improved from inf to 0.33837, saving model to /dbfs/user/odl_user_195841@databrickslabs.onmicrosoft.com/deep_learning/dl_03_advanced_keras/keras_checkpoint_weights.ckpt
WARNING:tensorflow:From /databricks/python/lib/python3.7/site-packages/tensorflow/python/ops/resource_variable_ops.py:1817: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /dbfs/user/odl_user_195841@databrickslabs.onmicrosoft.com/deep_learning/dl_03_advanced_keras/keras_checkpoint_weights.ckpt/assets
413/413 - 2s - loss: 0.3072 - mse: 0.3072 - val_loss: 0.3384 - val_mse: 0.3384
Epoch 2/10

Epoch 00002: val_loss improved from 0.33837 to 0.31146, saving model to /dbfs/user/odl_user_195841@databrickslabs.onmicrosoft.com/deep_learning/dl_03_advanced_keras/keras_checkpoint_weights.ckpt
INFO:tensorflow:Assets written to: /dbfs/user/odl_user_195841@databrickslabs.onmicrosoft.com/deep_learning/dl_03_advanced_keras/keras_checkpoint_weights.ckpt/assets
413/413 - 2s - loss: 0.3094 - mse: 0.3094 - val_loss: 0.3115 - val_mse: 0.3115
Epoch 3/10

Epoch 00003: val_loss did not improve from 0.31146
413/413 - 1s - loss: 0.3033 - mse: 0.3033 - val_loss: 0.3224 - val_mse: 0.3224
Epoch 4/10

Epoch 00004: val_loss did not improve from 0.31146
413/413 - 1s - loss: 0.3035 - mse: 0.3035 - val_loss: 0.3233 - val_mse: 0.3233
Epoch 5/10

Epoch 00005: val_loss did not improve from 0.31146
413/413 - 1s - loss: 0.3028 - mse: 0.3028 - val_loss: 0.3291 - val_mse: 0.3291
Epoch 6/10

Epoch 00006: val_loss did not improve from 0.31146
413/413 - 1s - loss: 0.2967 - mse: 0.2967 - val_loss: 0.3177 - val_mse: 0.3177
Epoch 7/10

Epoch 00007: val_loss did not improve from 0.31146
413/413 - 1s - loss: 0.3166 - mse: 0.3166 - val_loss: 0.3390 - val_mse: 0.3390
Epoch 8/10

Epoch 00008: val_loss did not improve from 0.31146
413/413 - 1s - loss: 0.2964 - mse: 0.2964 - val_loss: 0.3152 - val_mse: 0.3152
Epoch 9/10

Epoch 00009: val_loss did not improve from 0.31146
413/413 - 1s - loss: 0.2902 - mse: 0.2902 - val_loss: 0.3888 - val_mse: 0.3888
Epoch 10/10

Epoch 00010: val_loss did not improve from 0.31146
413/413 - 1s - loss: 0.2894 - mse: 0.2894 - val_loss: 0.3116 - val_mse: 0.3116

In [21]:
model.evaluate(X_test, y_test)

##validation loss is a bit higher than training. Model may be overfit

1/129 [..............................] - ETA: 0s - loss: 0.1623 - mse: 0.1623 55/129 [===========>..................] - ETA: 0s - loss: 0.3014 - mse: 0.3014108/129 [========================>.....] - ETA: 0s - loss: 0.3049 - mse: 0.3049129/129 [==============================] - 0s 929us/step - loss: 0.3052 - mse: 0.3052
Out[17]: [0.30519285798072815, 0.30519285798072815]

-sandbox
## 5. Dropout Regularization

It's generally more difficult to overtrain neural networks than classical machine learning methods.  However, overfitting is more common with smaller datasets and is caused in part by co-adapted neurons.

**Dropout is a regularization method that reduces overfitting by randomly and temporarily removing nodes during training.**  It works like this:<br><br>

- Apply to most type of layers (e.g. fully connected, convolutional, recurrent) and larger networks
- Set an additional probability for keeping each node
  - .5 is a good starting place for hidden layers
  - .9 is a good starting place for the input layer
- Temporarily and randomly remove nodes and their connections during each training cycle
- Since this results in larger weights, scale the weights proportional to the dropout rate
- Score on the entire architecture (without dropout)

![](https://files.training.databricks.com/images/nn_dropout.png)

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> See the original paper here: [Dropout: A Simple Way to Prevent Neural Networks from
Overfitting](http://jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf)

Redefine a larger network.  Note the following changes:<br><br>

- We'll use a dropout rate of .5
- Increase the size of the network in proportion to the dropout rate (`original network size` / `dropout rate`)
- Large weight sizes can be a sign of an unstable network.  Manage this using a weight constraint to force the magnitude of all weights to be below a specific value.  Typical values for this constraint `c` are between 3 and 4.

In [24]:
import tensorflow as tf
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
tf.random.set_seed(42)

def create_dropout_model():
  model = Sequential()
  model.add(Dropout(.1))
  model.add(Dense(40, input_dim=8, activation="relu", kernel_constraint=max_norm(4))) #use between 3 or 4 as rule of thumb
  model.add(Dropout(.5))#dropping 50% of units
  model.add(Dense(40, activation="relu", kernel_constraint=max_norm(4)))
  model.add(Dropout(.5))
  model.add(Dense(1, activation="linear"))
  return model

#drop out can come before or after adding units

dropoutModel = create_dropout_model()

Compile the model with a learning rate increased by 1-2 orders of magnitude.

> "Although dropout alone gives significant improvements, using dropout along with maxnorm regularization, large decaying learning rates and high momentum provides a significant
boost over just using dropout"
> [- Dropout: A Simple Way to Prevent Neural Networks from
Overfitting](http://jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf)

In [26]:
dropoutModel.compile(optimizer=Adam(lr=0.1), loss="mse", metrics=["mse"]) 

dropoutHistory = dropoutModel.fit(X_train, y_train, validation_split=.2, epochs=10, verbose=2)

Epoch 1/10
WARNING:tensorflow:Layer dropout is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2. The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

413/413 - 1s - loss: 1.6315 - mse: 1.6315 - val_loss: 1.3789 - val_mse: 1.3789
Epoch 2/10
413/413 - 1s - loss: 1.3550 - mse: 1.3550 - val_loss: 1.3439 - val_mse: 1.3439
Epoch 3/10
413/413 - 1s - loss: 1.3470 - mse: 1.3470 - val_loss: 1.3537 - val_mse: 1.3537
Epoch 4/10
413/413 - 1s - loss: 1.3606 - mse: 1.3606 - val_loss: 1.3437 - val_mse: 1.3437
Epoch 5/10
413/413 - 1s - loss: 1.3470 - mse: 1.3470 - val_loss: 1.3485 - val_mse: 1.3485
Epoch 6/10
413/413 - 1s - loss: 1.3502 - mse: 1.3502 - val_loss: 1.3589 - val_mse: 1.3589
Epoch 7/10
413/413 - 1s - loss: 1.3470 - mse: 1.3470 - val_loss: 1.3746 - val_mse: 1.3746
Epoch 8/10
413/413 - 1s - loss: 1.3490 - mse: 1.3490 - val_loss: 1.3421 - val_mse: 1.3421
Epoch 9/10
413/413 - 1s - loss: 1.5685 - mse: 1.5685 - val_loss: 1.3375 - val_mse: 1.3375
Epoch 10/10
413/413 - 1s - loss: 1.3442 - mse: 1.3442 - val_loss: 1.3355 - val_mse: 1.3355

-sandbox
<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> Batch normalization is an alternative approach.  This will be discussed in the lesson on CNNs.

Now it's your turn to try out these techniques on the Boston Housing Dataset!

-sandbox
&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>